## 1. 使用分组值填充缺失值
- 对于有缺失值的数据，可以使用`fillna()`使用指定的数据去填充缺失值
- 对于分组后的各组数据里的缺失值，如果需要使用与所属分组有关的数据来填充缺失值，可以使用`apply()`调用一个填充缺失值功能的函数来处理
- 也可以使用指定的值去填充不同分组的缺失值

In [1]:
import numpy as np
import pandas as pd

In [2]:
s=pd.Series(np.random.randn(10))
s[[2,5,8,9]]=np.nan
s

0    1.861629
1    0.571884
2         NaN
3   -0.385672
4    1.597879
5         NaN
6    0.757369
7   -1.056566
8         NaN
9         NaN
dtype: float64

In [3]:
# 使用平均值填充所有缺失值
s.fillna(s.mean())

0    1.861629
1    0.571884
2    0.557754
3   -0.385672
4    1.597879
5    0.557754
6    0.757369
7   -1.056566
8    0.557754
9    0.557754
dtype: float64

In [4]:
# 对数据根据索引进行奇偶分组
f=lambda x:'偶数' if x%2==0 else '奇数'
s.groupby(f).agg(['mean','count','sum'])

,mean,count,sum
偶数,1.405626,3,4.216877
奇数,-0.290118,3,-0.870354


In [5]:
# fillna_s：使用传入的数据的平均值来填充数据的缺失值
def fillna_s(x):
    return x.fillna(x.mean())

# 使用apply，将groupby后的每个分组分别传给fillna_s函数
# 即使用每个分组的平均值来填充该分组的缺失值
s.groupby(f).apply(fillna_s)

0    1.861629
1    0.571884
2    1.405626
3   -0.385672
4    1.597879
5   -0.290118
6    0.757369
7   -1.056566
8    1.405626
9   -0.290118
dtype: float64

In [6]:
# 为每个分组指定不同的缺失值
fill_value={'奇数':99,'偶数':88}
fna=lambda x:x.fillna(fill_value[x.name])
s.groupby(f).apply(fna)

0     1.861629
1     0.571884
2    88.000000
3    -0.385672
4     1.597879
5    99.000000
6     0.757369
7    -1.056566
8    88.000000
9    99.000000
dtype: float64

## 2.随机采样
目的：建立一副52张牌的扑克，每个花色随机抽取5张

In [7]:
card_suits=list('桃杏梅方')
card_letter={1:'A',11:'J',12:'Q',13:'K'}
# 建立A~K共13个字符格式的牌面值组成的列表
card_num=[card_letter[x] if x==1 or x>10 else str(x) for x in np.arange(1,14)]
cards=[]
# 建立4种花色共52张牌的列表
for suit in card_suits:
    cards.extend([num+suit for num in card_num])
cards=pd.Series(card_num*4,index=cards)
cards[:13]

A桃      A
2桃      2
3桃      3
4桃      4
5桃      5
6桃      6
7桃      7
8桃      8
9桃      9
10桃    10
J桃      J
Q桃      Q
K桃      K
dtype: object

In [8]:
# 随机获取5张牌
def draw(cards,n=5):
    return cards.sample(n)
draw(cards)

2方    2
4方    4
A梅    A
8梅    8
2桃    2
dtype: object

In [10]:
# groupby()里的函数是对索引进行处理的
# 根据索引的最后字符即花色进行分组，将每个分组代入draw()中随机抽取指定张数的牌
cards.groupby(lambda x:x[-1]).apply(draw,3)

方  10方    10
   9方      9
   5方      5
杏  4杏      4
   5杏      5
   2杏      2
桃  K桃      K
   7桃      7
   A桃      A
梅  4梅      4
   J梅      J
   Q梅      Q
dtype: object

## 3. 列与列之间运算
有时候分组后的聚合运算，需要根据列之间的关系同时使用多列进行运算处理，可以将运算方法建立为一个函数，然后使用apply()调用该函数，将分组结果分别代入该函数进行运算，最后运算结果再重新组合返回。  
**例：分组加权平均数**  
根据category进行分组并计算每组的加权平均数，data列为数据列，weights为加权列

In [11]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8), 'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.367151,0.103613
1,a,0.170277,0.538522
2,a,-0.342342,0.905314
3,a,1.580552,0.580276
4,b,0.255581,0.681119
5,b,1.005674,0.512413
6,b,1.198021,0.812152
7,b,-1.121427,0.106908


In [44]:
# 使用apply()调用average()函数，将数据和权重代入average()函数中
# 分组结果的每个片段都代入apply()所调用的average()函数中运算，将运算结果重新组合后返回
df.groupby('category').apply(lambda x:np.average(x['data'], weights=x['weights']))

category
a    0.310606
b    0.730139
dtype: float64